In [3]:
!python data/data_process.py --file_path data/Beauty/reviews_Beauty_5.json.gz --output_path data/Beauty

198502it [00:02, 93826.65it/s]
100%|█████████████████████████████████| 22363/22363 [00:00<00:00, 279691.14it/s]


In [ ]:
!bash script/train.sh 0 Beauty large_gpu